In [2]:
import json
import numpy as np
import os
import pandas as pd
import pickle
import random
import sys

directory = os.path.join(os.getcwd(), '../../../../Service\service\eeg')
# print(os.path.abspath(directory))

sys.path.append(directory)
from eeg_data_check import Filter, FFTDataCheck

#### 读取学生数据，排除掉 中/重度焦虑的学生

In [3]:
# 读取 Excel 文件
file_path = '../../../Data/healthy_scale_results.xlsx'  # 将此替换为您的 Excel 文件路径
df = pd.read_excel(file_path, sheet_name='BAI')

valid_cols = df.columns.values[5::]
print("valid columns: ", valid_cols)

df['score'] = df[valid_cols].sum(axis=1)
# print(df[['ID', 'score']])

invalid_negative_subjects_BAI = df[df['score'] >= 16]['ID'].tolist()
print("BAI invalid subjects: ", invalid_negative_subjects_BAI)

valid_negative_subjects_BAI = df[df['score'] < 16]['ID'].tolist()
print("BAI valid subjects: ", valid_negative_subjects_BAI)

valid columns:  ['1.身体麻木或刺痛' '2.感到发热' '3.腿部颤抖' '4.不能放松' '5.害怕要发生不好的事情' '6.感到头晕或眩晕'
 '7.心悸或心率加快' '8.心神不宁' '9.感到惊吓' '10.紧张' '11.有窒息感' '12.手发抖' '13.摇晃'
 '14.害怕失控' '15.呼吸困难' '16.害怕快要死去' '17.感到恐慌' '18.消化不良或腹部不适' '19.昏厥' '20.脸发红'
 '21.出汗（不是因为天气热）']
BAI invalid subjects:  ['A32', 'A41', 'A42', 'A48']
BAI valid subjects:  ['A33', 'A34', 'A35', 'A36', 'A37', 'A38', 'A39', 'A40', 'A43', 'A44', 'A45', 'A46', 'A47', 'A49', 'A50', 'A51', 'A52', 'A53', 'A54', 'A55']


In [3]:
sample_seconds = 0.9*1  # Each sample window seconds
sample_freq = 250 # 250hz 

sample_length = int(sample_freq * sample_seconds)


print(f"sample length: {sample_length}")

sample length: 225


In [4]:
# positive_subject_dir = './unhealthy'
# negative_subject_dir = './healthy'

positive_subject_dir = r"C:\Users\Administrator\Desktop\树兰医院&浙大_数据采集\EEG_Dataset_ALL\unhealthy"
negative_subject_dir = r"C:\Users\Administrator\Desktop\树兰医院&浙大_数据采集\EEG_Dataset_ALL\healthy"

In [5]:
# invalid_negative_ones = ['A03','A28','A17','A15','A12','A16','A10']
invalid_negative_ones = ['A32', 'A41', 'A42'] + invalid_negative_subjects_BAI

In [6]:

def get_negative_subjects(directory):
    try:
        # 获取文件夹下所有文件和文件夹的名字
        all_files = os.listdir(directory)
        # 过滤出以 'eeg' 开头的文件
        eeg_files = [file for file in all_files if file.endswith('.json')]
        
        valid_subjects = []
        seen = set()
        for file in eeg_files:
            subject = file.split('_')[0]
            if subject not in seen and subject not in invalid_negative_ones:
                valid_subjects.append(subject)
            seen.add(subject)
        return valid_subjects
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

# 使用示例
negative_subjects = get_negative_subjects(negative_subject_dir)
print(negative_subjects)

['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30', 'A31', 'A33', 'A34', 'A35', 'A36', 'A37', 'A38', 'A39', 'A40', 'A43', 'A44', 'A45', 'A46', 'A47', 'A49', 'A50', 'A51', 'A52', 'A53', 'A54', 'A55']


In [7]:
print("negative subject count: ", len(negative_subjects))

negative subject count:  51


In [6]:
invalid_postive_subjects = ['2A27', '2A33']

In [4]:
label_filepath = r'../../../Data/unhealthy_label.txt'

In [9]:
def get_positive_subjects(lable_filepath):
    userids = []
    try:
        with open(lable_filepath, 'r') as file:
            for line in file:
                parts = line.strip().split(',')
                if len(parts) == 3 and parts[2] == '1' and parts[0] not in invalid_postive_subjects:
                    userids.append(parts[0])
    except Exception as e:
        print(f"An error occurred: {e}")
    
    return userids

positive_subjects = get_positive_subjects(label_filepath)

In [11]:
random.shuffle(positive_subjects)
random.shuffle(negative_subjects)

print(f"positive subjects count: {len(positive_subjects)}", positive_subjects)
print(f"negative subjects count: {len(negative_subjects)}", negative_subjects)

positive subjects count: 27 ['20240717_2A07', '20240619_2A03', '20240619_2A05', '20240710_2A38', '20240710_2A02', '20240703_2A09', '20240717_2A08', '20240619_2A08', '20240619_2A26', '20240703_2A50', '20240703_2A17', '20240619_2A33', '20240626_2A26', '20240710_2A06', '20240717_2A51', '20240710_2A13', '20240710_2A15', '20240626_2A51', '20240619_2A31', '20240717_2A21', '20240619_2A17', '20240717_2A37', '20240710_2A23', '20240717_2A03', '20240710_2A30', '20240619_2A06', '20240703_2A38']
negative subjects count: 51 ['A05', 'A47', 'A55', 'A36', 'A02', 'A45', 'A34', 'A04', 'A15', 'A29', 'A16', 'A03', 'A26', 'A07', 'A37', 'A08', 'A31', 'A13', 'A53', 'A01', 'A54', 'A35', 'A33', 'A30', 'A11', 'A46', 'A17', 'A40', 'A38', 'A25', 'A52', 'A19', 'A24', 'A20', 'A14', 'A44', 'A23', 'A43', 'A18', 'A06', 'A09', 'A22', 'A12', 'A27', 'A39', 'A21', 'A50', 'A28', 'A49', 'A10', 'A51']


In [12]:
test_count = 8

In [13]:
# train_positive_subjects = positive_subjects[0: len(positive_subjects)-test_count]
# test_positive_subjects = positive_subjects[len(positive_subjects)-test_count::]

test_positive_subjects = ['20240619_2A05', '20240619_2A06', '20240626_2A51', '20240710_2A02', '20240710_2A13', '20240710_2A38' , '20240717_2A08', '20240717_2A37']
train_positive_subjects = [subject for subject in positive_subjects if subject not in test_positive_subjects]



test_negative_subjects_collection = ['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30']
random.shuffle(test_negative_subjects_collection)

test_negative_subjects = test_negative_subjects_collection[0:test_count]
train_negative_subjects = [subject for subject in negative_subjects if subject not in test_negative_subjects]

print("train positive subjects: ", train_positive_subjects)
print("test positive subjects: ", test_positive_subjects)

print("train negative subjects: ", train_negative_subjects)
print("test negative subjects: ", test_negative_subjects)

train positive subjects:  ['20240717_2A07', '20240619_2A03', '20240703_2A09', '20240619_2A08', '20240619_2A26', '20240703_2A50', '20240703_2A17', '20240619_2A33', '20240626_2A26', '20240710_2A06', '20240717_2A51', '20240710_2A15', '20240619_2A31', '20240717_2A21', '20240619_2A17', '20240710_2A23', '20240717_2A03', '20240710_2A30', '20240703_2A38']
test positive subjects:  ['20240619_2A05', '20240619_2A06', '20240626_2A51', '20240710_2A02', '20240710_2A13', '20240710_2A38', '20240717_2A08', '20240717_2A37']
train negative subjects:  ['A05', 'A47', 'A55', 'A36', 'A02', 'A45', 'A34', 'A04', 'A15', 'A29', 'A26', 'A07', 'A37', 'A08', 'A31', 'A13', 'A53', 'A01', 'A54', 'A35', 'A33', 'A30', 'A11', 'A46', 'A40', 'A38', 'A25', 'A52', 'A19', 'A24', 'A14', 'A44', 'A43', 'A06', 'A09', 'A27', 'A39', 'A21', 'A50', 'A28', 'A49', 'A10', 'A51']
test negative subjects:  ['A20', 'A23', 'A16', 'A03', 'A22', 'A17', 'A18', 'A12']


In [14]:

data_filter = FFTDataCheck()

In [15]:
len(train_negative_subjects)

43

In [16]:
train_positive_subjects

['20240717_2A07',
 '20240619_2A03',
 '20240703_2A09',
 '20240619_2A08',
 '20240619_2A26',
 '20240703_2A50',
 '20240703_2A17',
 '20240619_2A33',
 '20240626_2A26',
 '20240710_2A06',
 '20240717_2A51',
 '20240710_2A15',
 '20240619_2A31',
 '20240717_2A21',
 '20240619_2A17',
 '20240710_2A23',
 '20240717_2A03',
 '20240710_2A30',
 '20240703_2A38']

In [17]:
def read_json_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            # print("reading json file: ", file_path)
            data = json.load(file)
        return data
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def _get_samples(base_dir: str, target_subjects: list, label: int, data_aug: float=1.0):
    samples_ret = []
    for subject in target_subjects:
        files = os.listdir(base_dir)
        for file in files:
            if file.startswith(subject) and file.endswith('.json'):
                file_path = os.path.join(base_dir, file)
                json_data = read_json_file(file_path)
                left_wave_all, right_wave_all = [], []
                for raw_data in json_data['ProcessedDatas']:
                    left_wave = raw_data['LeftWave']
                    right_wave = raw_data['RightWave']

                    left_wave_all += left_wave
                    right_wave_all += right_wave


                if(len(left_wave_all) != len(right_wave_all)):
                    raise("error!!! left wave count not equal right wave count")
                
                samples = []
                step = int(sample_length / data_aug)

                # print(f"step: {step}")
                for idx in range(0, len(left_wave_all), step):
                        # print("idx: ", idx)
                        sub_left_wave = left_wave_all[idx:idx+sample_length]
                        sub_right_wave = right_wave_all[idx:idx+sample_length]
                        if len(sub_left_wave) != sample_length or len(sub_right_wave) != sample_length:
                            continue

                        left_valid = data_filter.is_valid(sub_left_wave)
                        right_valid = data_filter.is_valid(sub_right_wave)

                        # print("left_valid: ", left_valid, " right_valid: ", right_valid)
                        if left_valid and right_valid:
                            left_fft = np.abs(np.fft.fft(sub_left_wave))[:64]
                            left_fft = left_fft / np.sum(left_fft)

                            right_fft = np.abs(np.fft.fft(sub_right_wave))[:64]
                            right_fft = right_fft / np.sum(right_fft)

                            feat = np.concatenate([left_fft, right_fft], axis=0)
                            samples.append((subject, feat, label))
                samples_ret += samples
                print(f"read file: {file_path} left_wave_all: {len(left_wave_all)} sample length: {sample_length} data aug: {data_aug} step: {step}, samples count: {len(samples)}")
                
    return samples_ret


def get_train_negative_samples():
    return _get_samples(base_dir=negative_subject_dir, target_subjects=train_negative_subjects, label=0, data_aug=2)
    return _get_samples(base_dir=negative_subject_dir, target_subjects=train_negative_subjects, label=0, data_aug=2)

def get_test_negative_samples():
    return _get_samples(base_dir=negative_subject_dir, target_subjects=test_negative_subjects, label=0)


def get_train_positive_samples():
    return _get_samples(base_dir=positive_subject_dir, target_subjects=train_positive_subjects, label=1, data_aug =8)

def get_test_positive_samples():
    return _get_samples(base_dir=positive_subject_dir, target_subjects=test_positive_subjects, label=1)


train_negative_samples = get_train_negative_samples()
test_negative_samples = get_test_negative_samples()

train_positive_samples = get_train_positive_samples()
test_positive_samples = get_test_positive_samples()


read file: C:\Users\Administrator\Desktop\树兰医院&浙大_数据采集\EEG_Dataset_ALL\healthy\A05_EEG.json left_wave_all: 156600 sample length: 225 data aug: 2 step: 112, samples count: 1390
read file: C:\Users\Administrator\Desktop\树兰医院&浙大_数据采集\EEG_Dataset_ALL\healthy\A47_1.json left_wave_all: 126900 sample length: 225 data aug: 2 step: 112, samples count: 1122
read file: C:\Users\Administrator\Desktop\树兰医院&浙大_数据采集\EEG_Dataset_ALL\healthy\A47_2.json left_wave_all: 138600 sample length: 225 data aug: 2 step: 112, samples count: 1191
read file: C:\Users\Administrator\Desktop\树兰医院&浙大_数据采集\EEG_Dataset_ALL\healthy\A47_3.json left_wave_all: 143550 sample length: 225 data aug: 2 step: 112, samples count: 1273
read file: C:\Users\Administrator\Desktop\树兰医院&浙大_数据采集\EEG_Dataset_ALL\healthy\A55_2.json left_wave_all: 57600 sample length: 225 data aug: 2 step: 112, samples count: 494
read file: C:\Users\Administrator\Desktop\树兰医院&浙大_数据采集\EEG_Dataset_ALL\healthy\A55_2_02.json left_wave_all: 158850 sample length: 

In [18]:
print(f"len(train_negative_samples): {len(train_negative_samples)}")

len(train_negative_samples): 135101


In [19]:
print("train_negative_samples count: ", len(train_negative_samples))
print("train_positive_samples count: ", len(train_positive_samples))

print("test_negative_samples count: ", len(test_negative_samples))
print("test_positive_samples count: ", len(test_positive_samples))


# MIN = min(len(test_negative_samples), len(test_positive_samples))
# test_negative_samples = test_negative_samples[0:MIN]
# test_positive_samples = test_positive_samples[0:MIN]

print("\nAfter truncate: ")
print("test_negative_samples count: ", len(test_negative_samples))
print("test_positive_samples count: ", len(test_positive_samples))


train_negative_samples count:  135101
train_positive_samples count:  97393
test_negative_samples count:  5444
test_positive_samples count:  4569

After truncate: 
test_negative_samples count:  5444
test_positive_samples count:  4569


In [20]:
sample_folder = './samples'
if not os.path.exists(sample_folder):
    os.makedirs(sample_folder)

with open(os.path.join(sample_folder, 'train_negative_samples.pkl'), 'wb') as f:
    pickle.dump(train_negative_samples, f)

with open(os.path.join(sample_folder, 'train_positive_samples.pkl'), 'wb') as f:
    pickle.dump(train_positive_samples, f)

with open(os.path.join(sample_folder, 'test_negative_samples.pkl'), 'wb') as f:
    pickle.dump(test_negative_samples, f)

with open(os.path.join(sample_folder, 'test_positive_samples.pkl'), 'wb') as f:
    pickle.dump(test_positive_samples, f)


In [21]:
print(len(train_negative_samples[0][1]))
print(len(train_positive_samples[0][1]))
print(len(test_negative_samples[0][1]))
print(len(test_positive_samples[0][1]))

128
128
128
128
